In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from scipy.stats import uniform, randint

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [9]:
df = pd.read_csv('Данные для построения модели, задача - Регрессия для CC50.csv')
df

,EState_VSA3,PEOE_VSA9,BalabanJ,VSA_EState5,Chi2n,HallKierAlpha,Chi1v,Chi0n,qed,BertzCT,...,MolMR,LabuteASA,BCUT2D_CHGHI,EState_VSA9,Kappa1,BCUT2D_MWLOW,BCUT2D_MRLOW,VSA_EState2,fr_ketone_Topliss,Target Feature
0,21.659962,0.000000,1.164038,1.764908,12.058078,-0.66,12.204226,19.534409,0.417362,611.920301,...,121.5300,173.630124,2.600532,9.984809,20.606247,9.700470,0.193481,10.188192,0,2.244234
1,21.659962,0.000000,1.080362,1.914377,12.648545,-0.08,12.595754,19.794682,0.462473,516.780124,...,120.5074,174.939204,2.614066,0.000000,21.163454,9.689226,0.120322,0.000000,0,0.732620
2,21.659962,53.363882,1.219066,1.930720,15.671216,-0.08,14.249005,23.689110,0.260923,643.620154,...,138.4528,201.238858,2.665274,0.000000,25.026112,9.681293,-0.922902,0.000000,0,2.207210
3,21.659962,0.000000,1.120513,1.769975,12.411631,-0.66,12.704226,20.241516,0.377846,626.651366,...,126.1470,179.995066,2.600529,9.984809,21.567454,9.700497,0.193510,10.194720,0,2.032843
4,10.829981,11.374773,1.136678,1.605178,13.613700,-2.22,13.868825,22.617677,0.429038,1101.164252,...,148.3380,211.919602,2.602486,9.984809,23.194917,9.700386,0.193687,10.301020,0,2.143861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,11.835812,24.360600,1.646946,-0.405436,11.409461,-1.65,11.665192,18.825334,0.382752,857.600295,...,109.8350,178.490760,2.726237,9.473726,20.263719,9.344314,-0.196385,38.515343,0,1.544064
997,5.917906,18.945610,1.535171,-1.300354,12.636189,-2.18,13.274017,21.810933,0.369425,1016.917688,...,127.4397,207.296970,2.725543,9.473726,24.511583,9.343622,-0.187625,51.516353,0,1.531471
998,12.338728,18.945610,1.493776,-0.487671,13.248561,-1.83,15.695685,23.633394,0.284923,1070.961298,...,144.7647,230.149965,2.725818,9.473726,27.726151,9.343613,-0.187687,52.527620,0,1.531472
999,19.262465,51.104983,2.325807,-6.848660,11.893254,-2.45,13.174959,23.380977,0.381559,957.299494,...,131.7080,218.836986,2.730109,18.947452,29.111081,9.364015,-0.225309,67.303382,1,1.518509


In [3]:
X, y = df.drop('Target Feature', axis=1), df['Target Feature']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Target Feature', axis=1), df['Target Feature'], test_size=0.3, random_state=42)

#### Контрольной метрика для задачи регрессии будем считать R^2 score. Она будет записана в атрибут assessment_criterion BestRegressionModel класса

In [5]:
class BestRegressionModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели регресии
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'R^2',
            'value': 0.5
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LinearRegression(),
                'best_model': None,
                'best_params': None,
                'params': {},
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestRegressor(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': ElasticNet(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'alpha': [0.01, 0.1, 1, 10],
                    'l1_ratio': [0.2, 0.5, 0.8]
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVR(),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            self.models[model]['metrics']['MSE'] = mean_squared_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['MAE'] = mean_absolute_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['R^2'] = r2_score(y, self.models[model]['data']['y_pred'])
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"MSE: {self.models[model]['metrics']['MSE']:.4f}")
            print(f"MAE: {self.models[model]['metrics']['MAE']:.4f}")
            print(f"R^2: {self.models[model]['metrics']['R^2']:.4f}")
            print()
            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return 10 ** result

#### Подбор гипперпараметров для моделей

In [ ]:
best_regression_model = BestRegressionModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LinearRegression() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели LinearRegression(): {}

Подбор гипперпараметров для модели RandomForestRegressor(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestRegressor(random_state=42): {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 141}

Подбор гипперпараметров для модели ElasticNet(random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели ElasticNet(random_state=42): {'alpha': 0.01, 'l1_ratio': 0.5}

Подбор гипперпараметров для модели SVR() методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию пара

#### Поиск лучшей модели

In [7]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.3589
MAE: 0.4952
R^2: 0.2065

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.2895
MAE: 0.3983
R^2: 0.3599

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.3595
MAE: 0.4935
R^2: 0.2052

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

#### Формирование предсказания лучшей моделью

In [8]:
best_regression_model.predict(X_test)

Модель для формирования прогноза не сформирована, используйте метод get_best_model


Так как мы не смогли найти модель, которая была бы лучше, чем заданное значение метрикик, изменим значение R^2 score на 0.3

In [10]:
best_regression_model.assessment_criterion['value'] = 0.3

Попробуем снова подобрать модель

In [11]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.3589
MAE: 0.4952
R^2: 0.2065

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.2895
MAE: 0.3983
R^2: 0.3599

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.3595
MAE: 0.4935
R^2: 0.2052

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

In [12]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestRegressor(min_samples_leaf=2, min_samples_split=5, n_estimators=141,
                      random_state=42)
Предсказание сформировано


array([ 789.59825382,  426.93844609,  111.51991336,   40.46704226,
        509.06270656,  331.56831492,  945.72763011,  205.75693836,
         51.57830428,  624.0506919 ,   25.36386159,   12.43276161,
        199.53010971,  150.13080896,   93.44635258,  104.81459628,
        101.71967445,    8.06344689,  979.43006812,   26.71822744,
        163.13954979,  567.47689043,  373.18219751,  728.95958645,
        525.65570849,  227.49920182, 3152.34144932,  133.74164187,
        101.4400533 ,   51.57830428,  326.28016822,  381.92214479,
        764.51619197,  351.6714792 ,  580.56101833,  515.49240458,
        471.18343821,   27.12251667,  327.38264496,  716.63596392,
         28.10249293,  716.63596392,   41.11676277,   91.70075195,
        166.6111803 ,  175.0421876 ,  122.54698775,  321.02578189,
        376.83573397,  430.90522992,  171.65466572,   10.30668477,
        967.34018982,  225.68762123,  980.61396646,  525.82262823,
        979.44663915,  137.4809458 ,  183.07347762,   93.16741